[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BSDExabio/SAFA/blob/main/StructuralAlignmentWorkflow/usalign_parser.ipynb)

# Overview of the `usalign_parser` functions
The functions in the `usalign_parser` module file were developed to gather the relevant quantitative information reported from US-align2 for specific use cases that are presented in `./USalignParser/examples/automate_alnments.sh` and the `runUSalign*sh` bash scripts. One simple example of these bash scripts is shown below:

```bash
USALIGN_HOME=~/Apps/USalign/
PDB_PATH=./USalignParser/examples/
$USALIGN_HOME/USalign $PDB_PATH/3cna_rcsb.pdb $PDB_PATH/2pel_rcsb.pdb -ter 2 -m temp.dat -outfmt 0 -mm 6 
cat temp.dat
rm temp.dat
```

The intent for these bash scripts are to perform a structural alignment between two single-chain protein structures and gather all relevant information, including the alignment mapping between the two structures (when provided). US-align2 parameters set in these bash scripts are `-ter 2` (only align the first chain in both structure files), ``-m temp.dat`` (write the translation and rotation matrix to file `temp.dat`), and `-outfmt 0` (write the log file in original format). All other parameters are using default values. 

For these bash scripts, the US-align2 output (`-outfmt 0`) is printed to standard out. This is specifically done because these scripts are used in `subprocess.run` calls within the dask client script, where the standard out is gathered without any IO to storage space. Unfortunately, the designed behavior of US-align2 for reporting the translation and rotation matrix (`-m` parameter) is to write that data to file. To work around this, a temporary file is written, containing the translation and rotation matrix, that is subsequently concatenated to the bottom of the standard out and deleted. This is done within the bash scripts so the standard out variable from the `subprocess.run` contains all important output from the US-align2 calculation.

Finally, the `-mm` parameter is set to different values depending on the alignment algorithm to be used. There are four `-mm` settings that are most relevant to our use-case: `0` sets the sequential (SQ) alignment algorithm,`3` sets the circular permutation (CP) algorithm, `5` sets the fully-non-sequential (fNS) algorithm, and `6` sets the semi-non-sequential (sNS) algorithm. All four settings share the same format for the majority of the output/log file, specifically where alignment quality metrics are reported. Importantly, the CP, fNS, and sNS methods also report the alignment mapping between target and query structures.


In [2]:
from USalignParser.usalign_parser import parse_usalign_file, threshold_comparison

## Parsing US-align2 alignment result files
The ``usalign_parser.parse_usalign_file`` function gathers all relevant information from the above described methods' log files and returns a data structure for further analysis. Below, the example files are parsed and the resulting objects are presented for consideration. 

### Sequentially-dependent Alignment Algorithm

Without a translation and rotation matrix written to the log file:

In [3]:
log_file = './USalignParser/examples/example_output/3cnaA_to_2pelA_USalign_SQ_noTransRotMat.log'
with open(log_file,'r') as log:
    result_dict, start, stop, return_code = parse_usalign_file(log,'')
print(f"Parsing the log file took {stop-start} with return code {return_code}.\n")

print(f"query structure:  {result_dict['struct1']}: chain {result_dict['struct1_chainID']}, length: {result_dict['Len1']} residues")
print(f"target structure: {result_dict['struct2']}: chain {result_dict['struct2_chainID']}, length: {result_dict['Len2']} residues\n")

print(f"Aligned length: {result_dict['LenAligned']}, RMSD = {result_dict['RMSD']} Angstroms, Sequence identity of aligned residues = {result_dict['SeqIDAli']}")
print(f"TM-score1 = {result_dict['TMscore1']}, normalized by query  length and d0 = {result_dict['d0_1']}")
print(f"TM-score2 = {result_dict['TMscore2']}, normalized by target length and d0 = {result_dict['d0_2']}")

print(f"The average TMscore is {(result_dict['TMscore1']+result_dict['TMscore2'])/2.}.")

'' not expected by parser function. Only returning alignment quantitative metrics. No mapping.


Parsing the log file took 0.007493734359741211 with return code 1.

query structure:  ./3cna_rcsb.pdb: chain A, length: 237.0 residues
target structure: ./2pel_rcsb.pdb: chain A, length: 232.0 residues

Aligned length: 118.0, RMSD = 1.63 Angstroms, Sequence identity of aligned residues = 0.432
TM-score1 = 0.47057, normalized by query  length and d0 = 5.71
TM-score2 = 0.48028, normalized by target length and d0 = 5.65
The average TMscore is 0.475425.


With a translation and rotation matrix written to the log file:

In [4]:
log_file = './USalignParser/examples/example_output/3cnaA_to_2pelA_USalign_SQ.log'
with open(log_file,'r') as log:
    result_dict, start, stop, return_code = parse_usalign_file(log,'')
print(f"Parsing the log file took {stop-start} with return code {return_code}.\n")

print(f"query structure:  {result_dict['struct1']}: chain {result_dict['struct1_chainID']}, length: {result_dict['Len1']} residues")
print(f"target structure: {result_dict['struct2']}: chain {result_dict['struct2_chainID']}, length: {result_dict['Len2']} residues\n")

print(f"Aligned length: {result_dict['LenAligned']}, RMSD = {result_dict['RMSD']} Angstroms, Sequence identity of aligned residues = {result_dict['SeqIDAli']}")
print(f"TM-score1 = {result_dict['TMscore1']}, normalized by query  length and d0 = {result_dict['d0_1']}")
print(f"TM-score2 = {result_dict['TMscore2']}, normalized by target length and d0 = {result_dict['d0_2']}")

print(f"The average TMscore is {(result_dict['TMscore1']+result_dict['TMscore2'])/2.}.\n")

print(f"Translational degrees of freedom vector: {result_dict['trans_vector']}")
print(f"Rotational degrees of freedom matrix: {result_dict['rot_matrix']}")

'' not expected by parser function. Only returning alignment quantitative metrics. No mapping.


Parsing the log file took 0.0057103633880615234 with return code 1.

query structure:  ./3cna_rcsb.pdb: chain A, length: 237.0 residues
target structure: ./2pel_rcsb.pdb: chain A, length: 232.0 residues

Aligned length: 118.0, RMSD = 1.63 Angstroms, Sequence identity of aligned residues = 0.432
TM-score1 = 0.47057, normalized by query  length and d0 = 5.71
TM-score2 = 0.48028, normalized by target length and d0 = 5.65
The average TMscore is 0.475425.

Translational degrees of freedom vector: [29.61731488  4.98254133 36.53334062]
Rotational degrees of freedom matrix: [[ 0.08032842 -0.11836715  0.98971539]
 [ 0.99614362 -0.02561858 -0.08391407]
 [ 0.03528777  0.99263936  0.11585278]]


In the latter example, the translation and rotational matrix was parsed. This data is necessary for recreating the alignment between the two structures.

For both, a return code of 1 means that no alignment mapping between the two structures was parsed.

### Circular Permutation Alignment Algorithm

In [5]:
log_file = './USalignParser/examples/example_output/3cnaA_to_2pelA_USalign_CP.log'
with open(log_file,'r') as log:
    result_dict, start, stop, return_code = parse_usalign_file(log,'CP')
print(f"Parsing the log file took {stop-start} with return code {return_code}.\n")

print(f"query structure:  {result_dict['struct1']}: chain {result_dict['struct1_chainID']}, length: {result_dict['Len1']} residues")
print(f"target structure: {result_dict['struct2']}: chain {result_dict['struct2_chainID']}, length: {result_dict['Len2']} residues\n")

print(f"Aligned length: {result_dict['LenAligned']}, RMSD = {result_dict['RMSD']} Angstroms, Sequence identity of aligned residues = {result_dict['SeqIDAli']}")
print(f"TM-score1 = {result_dict['TMscore1']}, normalized by query  length and d0 = {result_dict['d0_1']}")
print(f"TM-score2 = {result_dict['TMscore2']}, normalized by target length and d0 = {result_dict['d0_2']}")

print(f"The average TMscore is {(result_dict['TMscore1']+result_dict['TMscore2'])/2.}.\n")

print(f"Translational degrees of freedom vector: {result_dict['trans_vector']}")
print(f"Rotational degrees of freedom matrix: {result_dict['rot_matrix']}\n")

print(f"First elements in the alignment mapping:\n{[(key, value) for key, value in result_dict['map_1_to_2'].items()][:10]}")

Parsing the log file took 0.0063784122467041016 with return code 0.

query structure:  ./3cna_rcsb.pdb: chain A, length: 237.0 residues
target structure: ./2pel_rcsb.pdb: chain A, length: 232.0 residues

Aligned length: 228.0, RMSD = 1.93 Angstroms, Sequence identity of aligned residues = 0.408
TM-score1 = 0.89129, normalized by query  length and d0 = 5.71
TM-score2 = 0.90943, normalized by target length and d0 = 5.65
The average TMscore is 0.90036.

Translational degrees of freedom vector: [29.36918745  5.15412648 36.83881406]
Rotational degrees of freedom matrix: [[ 0.07836471 -0.10453358  0.99142912]
 [ 0.99629091 -0.02724568 -0.08162171]
 [ 0.03554437  0.99414808  0.10201076]]

First elements in the alignment mapping:
[('123', ('1', 'THR', 'ALA')), ('124', ('2', 'ASP', 'GLU')), ('125', ('3', 'ALA', 'THR')), ('126', ('4', 'LEU', 'VAL')), ('127', ('5', 'HIS', 'SER')), ('128', ('6', 'PHE', 'PHE')), ('129', ('7', 'MET', 'ASN')), ('130', ('8', 'PHE', 'PHE')), ('131', ('9', 'ASN', 'ASN')

The average TMscore calculated using the Circular Permutation alignment algorithm is now 0.90, compared to the 0.48 from the SQ algorithm. The CP algo also aligns 228 residues, representing nearly all residues in both target and query structures. 

Additionally, the CP log file also contains the mapping between query and target residues from the alignment. This mapping is key for transferal of residue-level information associated with either query or target structures. The mapping result is stored in the ``result_dict['map_1_to_2']`` dictionary object, where keys are residue index numbers for the target structure (structure_2) and values are tuples with organization (query structure residue index, target residue name, query residue name).

### fully-non-sequential (fNS) Alignment Algorithm

In [6]:
log_file = './USalignParser/examples/example_output/3cnaA_to_2pelA_USalign_fNS.log'
with open(log_file,'r') as log:
    result_dict, start, stop, return_code = parse_usalign_file(log,'fNS')
print(f"Parsing the log file took {stop-start} with return code {return_code}.\n")

print(f"query structure:  {result_dict['struct1']}: chain {result_dict['struct1_chainID']}, length: {result_dict['Len1']} residues")
print(f"target structure: {result_dict['struct2']}: chain {result_dict['struct2_chainID']}, length: {result_dict['Len2']} residues\n")

print(f"Aligned length: {result_dict['LenAligned']}, RMSD = {result_dict['RMSD']} Angstroms, Sequence identity of aligned residues = {result_dict['SeqIDAli']}")
print(f"TM-score1 = {result_dict['TMscore1']}, normalized by query  length and d0 = {result_dict['d0_1']}")
print(f"TM-score2 = {result_dict['TMscore2']}, normalized by target length and d0 = {result_dict['d0_2']}")

print(f"The average TMscore is {(result_dict['TMscore1']+result_dict['TMscore2'])/2.}.\n")

print(f"Translational degrees of freedom vector: {result_dict['trans_vector']}")
print(f"Rotational degrees of freedom matrix: {result_dict['rot_matrix']}\n")

print(f"First elements in the alignment mapping:\n{[(key, value) for key, value in result_dict['map_1_to_2'].items()][:10]}")

Parsing the log file took 0.0051844120025634766 with return code 0.

query structure:  ./3cna_rcsb.pdb: chain A, length: 237.0 residues
target structure: ./2pel_rcsb.pdb: chain A, length: 232.0 residues

Aligned length: 228.0, RMSD = 1.9 Angstroms, Sequence identity of aligned residues = 0.408
TM-score1 = 0.89216, normalized by query  length and d0 = 5.71
TM-score2 = 0.91034, normalized by target length and d0 = 5.65
The average TMscore is 0.90125.

Translational degrees of freedom vector: [29.4250692   5.11414418 36.7811989 ]
Rotational degrees of freedom matrix: [[ 0.076364   -0.10631341  0.99139598]
 [ 0.9963703  -0.02937206 -0.0798969 ]
 [ 0.03761345  0.99389876  0.10368456]]

First elements in the alignment mapping:
[('123', ('1', 'THR', 'ALA', 0.494)), ('124', ('2', 'ASP', 'GLU', 0.75)), ('125', ('3', 'ALA', 'THR', 0.263)), ('126', ('4', 'LEU', 'VAL', 0.792)), ('127', ('5', 'HIS', 'SER', 0.629)), ('128', ('6', 'PHE', 'PHE', 0.733)), ('129', ('7', 'MET', 'ASN', 1.045)), ('130', ('

The average TMscore calculated using the fNS alignment algorithm is 0.90 with the same alignment length as the CP algo. 

Similar to above, the fNS log file contains the mapping between query and target residues from the alignment. But, the mapping now includes a distance metric between the aligned residues' CA atoms; the tuple is organized as (query structure residue index, target residue name, query residue name, interatomic distance in Angstroms).

### Semi-non-sequential (sNS) Alignment Algorithm

In [7]:
log_file = './USalignParser/examples/example_output/3cnaA_to_2pelA_USalign_sNS.log'
with open(log_file,'r') as log:
    result_dict, start, stop, return_code = parse_usalign_file(log,'sNS')
print(f"Parsing the log file took {stop-start} with return code {return_code}.\n")

print(f"query structure:  {result_dict['struct1']}: chain {result_dict['struct1_chainID']}, length: {result_dict['Len1']} residues")
print(f"target structure: {result_dict['struct2']}: chain {result_dict['struct2_chainID']}, length: {result_dict['Len2']} residues\n")

print(f"Aligned length: {result_dict['LenAligned']}, RMSD = {result_dict['RMSD']} Angstroms, Sequence identity of aligned residues = {result_dict['SeqIDAli']}")
print(f"TM-score1 = {result_dict['TMscore1']}, normalized by query  length and d0 = {result_dict['d0_1']}")
print(f"TM-score2 = {result_dict['TMscore2']}, normalized by target length and d0 = {result_dict['d0_2']}")

print(f"The average TMscore is {(result_dict['TMscore1']+result_dict['TMscore2'])/2.}.\n")

print(f"Translational degrees of freedom vector: {result_dict['trans_vector']}")
print(f"Rotational degrees of freedom matrix: {result_dict['rot_matrix']}\n")

print(f"First elements in the alignment mapping:\n{[(key, value) for key, value in result_dict['map_1_to_2'].items()][:10]}")

Parsing the log file took 0.000957489013671875 with return code 0.

query structure:  ./3cna_rcsb.pdb: chain A, length: 237.0 residues
target structure: ./2pel_rcsb.pdb: chain A, length: 232.0 residues

Aligned length: 228.0, RMSD = 1.93 Angstroms, Sequence identity of aligned residues = 0.408
TM-score1 = 0.89227, normalized by query  length and d0 = 5.71
TM-score2 = 0.91045, normalized by target length and d0 = 5.65
The average TMscore is 0.9013599999999999.

Translational degrees of freedom vector: [29.39467542  5.1576287  36.81035094]
Rotational degrees of freedom matrix: [[ 0.07966211 -0.10571238  0.99120071]
 [ 0.99616087 -0.02776471 -0.08302188]
 [ 0.03629684  0.99400906  0.10309474]]

First elements in the alignment mapping:
[('123', ('1', 'THR', 'ALA', 0.42)), ('124', ('2', 'ASP', 'GLU', 0.677)), ('125', ('3', 'ALA', 'THR', 0.254)), ('126', ('4', 'LEU', 'VAL', 0.749)), ('127', ('5', 'HIS', 'SER', 0.606)), ('128', ('6', 'PHE', 'PHE', 0.71)), ('129', ('7', 'MET', 'ASN', 1.052)), 

The alignment obtained from the sNS algorithm is of similar quality to the fNS alignment. The mapping has the same formatting as well. 

## Applying a metric cutoff to determine whether an alignment is high quality

When running a large number of alignments (we're talking thousands to billions), we do not want to save all results to file. Instead, the above ``parse_usalign_file`` function can be used to parse a string (as if it were  a file). Then, we can judge whether the alignment results interesting and should be written to storage. Below is an example:

In [8]:
# creating a string that mirrors those created during the SAFA alignment dask workflow. 
log_file = './USalignParser/examples/example_output/3cnaA_to_2pelA_USalign_sNS.log'
with open(log_file,'r') as log:
    log_string = log.read()
# this is the log file saved in a string object; in the workflow, the string object would 
# be the standard out from the subprocess.run of the US-align bash scripts
print(log_string)


 ********************************************************************
 * US-align (Version 20220924)                                      *
 * Universal Structure Alignment of Proteins and Nucleic Acids      *
 * Reference: C Zhang, M Shine, AM Pyle, Y Zhang. (2022) Nat Methods*
 * Please email comments and suggestions to yangzhanglab@umich.edu  *
 ********************************************************************

Name of Structure_1: ./3cna_rcsb.pdb:A (to be superimposed onto Structure_2)
Name of Structure_2: ./2pel_rcsb.pdb:A
Length of Structure_1: 237 residues
Length of Structure_2: 232 residues

Aligned length= 228, RMSD=   1.93, Seq_ID=n_identical/n_aligned= 0.408
TM-score= 0.89227 (normalized by length of Structure_1: L=237, d0=5.71)
TM-score= 0.91045 (normalized by length of Structure_2: L=232, d0=5.65)
(You should use TM-score normalized by length of the reference structure)

(":" denotes residue pairs of d < 5.0 Angstrom, "." denotes other aligned residues)
TDALHFMFNQFSKDQ

In [9]:
import io
# create the fake file object
log_file = io.StringIO(log_string)
# use the file object as input to the parse_usalign_file function
result_dict, start, stop, return_code = parse_usalign_file(log_file,'sNS')

print(result_dict['TMscore1'],result_dict['TMscore2'],result_dict['RMSD'])

0.89227 0.91045 1.93


### ``threshold_comparison`` function

The comparison of quantitative metric(s) can be performed to determine if an alignment is sufficiently high-quality to warrent further consideration. Numerous metrics can be used to make that decision. If an alignment's result passes the cutoff, the ``threshold_comparison`` function returns 1 (integer True); else, it will return 0 (integer False). 

In [10]:
threshold_value = 0.7
metric = 'avgtmscore'

threshold_bool = threshold_comparison(result_dict, metric, threshold_value)
if threshold_bool:
    print(f'{threshold_bool}, alignment does pass the cutoff of {threshold_value} on the {metric}')
else:
    print(f'{threshold_bool}, alignment does NOT pass the cutoff of {threshold_value} on the {metric}')
    
threshold_value = 0.99
threshold_bool = threshold_comparison(result_dict, metric, threshold_value)
if threshold_bool:
    print(f'{threshold_bool}, alignment does pass the cutoff of {threshold_value} on the {metric}')
else:
    print(f'{threshold_bool}, alignment does NOT pass the cutoff of {threshold_value} on the {metric}')

1, alignment does pass the cutoff of 0.7 on the avgtmscore
0, alignment does NOT pass the cutoff of 0.99 on the avgtmscore


Many metrics are currently handled, all focusing on the TMscore values from the alignment. Any capitalization of 'mintmscore', 'maxtmscore', 'tmscore1', 'tmscore2', 'avgtmscore'. 

In [11]:
threshold_value = 0.9

metric = 'mintmscore'
threshold_bool = threshold_comparison(result_dict, metric, threshold_value)
if threshold_bool:
    print(f'{threshold_bool}, alignment does pass the cutoff of {threshold_value} on the {metric}')
else:
    print(f'{threshold_bool}, alignment does NOT pass the cutoff of {threshold_value} on the {metric}')

metric = 'MaXtMsCoRe'
threshold_bool = threshold_comparison(result_dict, metric, threshold_value)
if threshold_bool:
    print(f'{threshold_bool}, alignment does pass the cutoff of {threshold_value} on the {metric}')
else:
    print(f'{threshold_bool}, alignment does NOT pass the cutoff of {threshold_value} on the {metric}')

metric = 'TMscore1'
threshold_bool = threshold_comparison(result_dict, metric, threshold_value)
if threshold_bool:
    print(f'{threshold_bool}, alignment does pass the cutoff of {threshold_value} on the {metric}')
else:
    print(f'{threshold_bool}, alignment does NOT pass the cutoff of {threshold_value} on the {metric}')

metric = 'TMscore2'
threshold_bool = threshold_comparison(result_dict, metric, threshold_value)
if threshold_bool:
    print(f'{threshold_bool}, alignment does pass the cutoff of {threshold_value} on the {metric}')
else:
    print(f'{threshold_bool}, alignment does NOT pass the cutoff of {threshold_value} on the {metric}')


0, alignment does NOT pass the cutoff of 0.9 on the mintmscore
1, alignment does pass the cutoff of 0.9 on the MaXtMsCoRe
0, alignment does NOT pass the cutoff of 0.9 on the TMscore1
1, alignment does pass the cutoff of 0.9 on the TMscore2


With the boolean value, IO of results can be gated for alignments that surpass the threshold.